In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

import os
import re

In [ ]:
reviews_train = []
for line in open('./data/imdb-review/full_train.txt', 'r',encoding='utf8'):
    reviews_train.append(line.strip())

reviews_test = []
for line in open('./data/imdb-review/full_test.txt', 'r',encoding='utf8'):
    reviews_test.append(line.strip())

In [ ]:
reviews_train[5]

In [ ]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

reviews_train_clean[5]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary = True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

target = [1 if i<12500 else 0 for i in range(25000)]

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C = c)
    lr.fit(X_train, y_train)
    print("Accuracy for C=%s : %s"%(c, accuracy_score(y_val, lr.predict(X_val))))

In [ ]:
final_model = LogisticRegression(C = 0.05)
final_model.fit(X, target)
print("Final Accuracy: %s" %accuracy_score(target, final_model.predict(X_test)))

In [ ]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(),
    key = lambda x: x[1],
    reverse = True)[:5]:
    print(best_positive)

In [ ]:
for best_negative in sorted(feature_to_coef.items(),
                           key = lambda x: x[1])[:5]:
    print(best_negative)